In [1]:
from pprint import pprint
import json

import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.documents as documents
import azure.cosmos.errors as errors

In [2]:
COSMOSDB_ENDPOINT = '{AZURE_COSMOS_DB_ENDPOINT}'
COSMOSDB_PRIMARY_KEY = '{AZURE_COSMOS_DB_PRIMARY_KEY}'
COSMOSDB_DB_ID = 'UniversityDatabase'
COSMOSDB_COLL_ID = 'StudentCollection'

client = cosmos_client.CosmosClient(COSMOSDB_ENDPOINT, { 'masterKey': COSMOSDB_PRIMARY_KEY })
database_link = f'dbs/{COSMOSDB_DB_ID}'
throughput = 11000
partition_key = '/enrollmentYear'
unique_key = '/studentAlias'
collection_link = f'{database_link}/colls/{COSMOSDB_COLL_ID}'

In [3]:
def create_database(id):
    print('Create database')

    try:
        client.CreateDatabase({"id": id})
        print(f'Database with id "{id}" created')

    except errors.HTTPFailure as e:
        if e.status_code == 409:
           print(f'A database with id "{id}" already exists')
        else: 
            raise

In [4]:
create_database(COSMOSDB_DB_ID)

Create database
Database with id "UniversityDatabase" created


In [5]:
def create_container(id, throughput, partition_key, unique_key):
    try:
        coll = {
            "id": id,
            "partitionKey": {
                "paths": [
                    partition_key
                ],
                "kind": "Hash",
                "version": 2
            },
            'uniqueKeyPolicy': {
                'uniqueKeys': [
                    { 'paths': [ unique_key ] }
                ]
            }
        }
        collection_options = { 'offerThroughput': throughput }
        collection = client.CreateContainer(database_link, coll, collection_options)

        print(f'Collection with id "{id}" created')
        print(f'Partition Key - "{partition_key}"')

    except errors.CosmosError as e:
        if e.status_code == 409:
            print(f'A collection with id "{id}" already exists')
        else:
            raise

In [6]:
create_container(COSMOSDB_COLL_ID, throughput, partition_key, unique_key)

Collection with id "StudentCollection" created
Partition Key - "/enrollmentYear"


In [7]:
def create_documents(docs):
    client.CreateItem(collection_link, docs)

In [8]:
# The upload may take up to 15 minutes to complete

with open('../data/students-25%.json') as json_file:
    students_json = json.load(json_file)
    for student in students_json:
        create_documents(student)

In [9]:
def query_documents(collection_link, query, options={ "enableCrossPartitionQuery": True }):
    try:
        results = list(client.QueryItems(collection_link, query, options))
        return results

    except errors.HTTPFailure as e:
        if e.status_code == 404:
            print("Document doesn't exist")
        elif e.status_code == 400:
            # Can occur when we are trying to query on excluded paths
            print("Bad Request exception occured: ", e)
            pass
        else:
            raise

    finally:
        print()

In [10]:
query = 'SELECT * FROM students s WHERE s.enrollmentYear = 2017'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2111
{'_attachments': 'attachments/',
 '_etag': '"07006432-0000-0100-0000-5e34673e0000"',
 '_rid': 'optQAMq6M+ACAAAAAAAACA==',
 '_self': 'dbs/optQAA==/colls/optQAMq6M+A=/docs/optQAMq6M+ACAAAAAAAACA==/',
 '_ts': 1580492606,
 'academicStatus': {'expulsion': False, 'suspension': False, 'warning': False},
 'age': 20,
 'clubs': ['MMA Club'],
 'enrollmentYear': 2017,
 'financialData': {'tuitionBalance': 0,
                   'tuitionPayments': [{'amount': 3200, 'date': '2017-01-13'},
                                       {'amount': 3400, 'date': '2016-07-08'},
                                       {'amount': 4000, 'date': '2017-03-18'},
                                       {'amount': 4000, 'date': '2017-09-22'},
                                       {'amount': 2900, 'date': '2017-09-01'}],
                   'unusedFinancialAid': 0},
 'firstName': 'Albin',
 'homeAddress': {'city': 'Jaedenhaven',
                 'numberAndStreet': '8352 Garfield Neck',
        

In [11]:
# In this query, we drop the 's' alias and use the 'students' source. When we execute this query, we should see the same results as the previous query.

query = 'SELECT * FROM students WHERE students.enrollmentYear = 2017'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')


Length of query results: 2111


In [12]:
# In this query, we will prove that the name used for the source can be any name you choose. We will use the name 'arbitraryname' for the source.
# When we execute this query, we should see the same results as the previous query.

query = 'SELECT * FROM arbitraryname WHERE arbitraryname.enrollmentYear = 2017'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')


Length of query results: 2111


In [13]:
# Going back to 's' as an alias, we will now create a query where we only select the 'studentAlias' property and return the value of
# that property in our result set.

query = 'SELECT s.studentAlias FROM students s WHERE s.enrollmentYear = 2017'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2111
{'studentAlias': 'albinvonrueden168071'}


In [14]:
# In some scenarios, you may need to return a flattened array as the result of your query. This query uses the 'VALUE' keyword to flatten
# the array by taking the single returned (string) property and creating a string array.

query = 'SELECT VALUE s.studentAlias FROM students s WHERE s.enrollmentYear = 2017'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results)


Length of query results: 2111
['albinvonrueden168071',
 'caliabbott079142',
 'maeterry067504',
 'mathildehartmann036171',
 'remingtondaugherty069221',
 'kennethtreutel003169',
 'angelrogahn065095',
 'yesseniathiel051729',
 'wilfredorosenbaum164029',
 'norbertojacobs016273',
 'olliethiel025164',
 'alvinagraham084904',
 'bennettsporer069310',
 'lessiedietrich096075',
 'elnaherzog170956',
 'elizabethshields151582',
 'nayelifranecki035212',
 'dameonpredovic105822',
 'beauframi066057',
 'dorthajakubowski138384',
 'maudiekertzmann025704',
 'annettakris072416',
 'janickweber104027',
 'tamiabins018592',
 'estelltrantow026638',
 'rogerkoss059505',
 'myronbartoletti022668',
 'kennedyhermann129585',
 'aldarath147774',
 'burleywunsch091282',
 'anyakihn156771',
 'ottiliejacobs015175',
 'marvingrady016483',
 'audreannewindler035930',
 'hopebogan141891',
 'lucilebrakus168276',
 'luellajacobs000373',
 'verliehayes144686',
 'jonasbogan076224',
 'mariamkreiger038801',
 'kolenitzsche092376',
 'lavernlan

 'joanagoyette115223',
 'reillylebsack121440',
 'chaimcummerata037736',
 'kaileelehner091769',
 'soniaglover050829',
 'tremaynenader117173',
 'bradleyhackett169657',
 'keatoncremin056893',
 'savanahjohns058814',
 'marcgraham007005',
 'jalenschulist069483',
 'savannahcormier084109',
 'germainelowe035923',
 'janickleannon035548',
 'brianward119022',
 'rosaliagerlach131276',
 'valentinking018963',
 'brannondoyle088253',
 'chanelbartoletti168485',
 'kendrickwaelchi007024',
 'laruemedhurst054912',
 'kaceykeeling060342',
 'rosariomcdermott096594',
 'lucianogottlieb110284',
 'karliconroy159449',
 'marcelgrady164939',
 'margaretewyman075070',
 'theresenikolaus013972',
 'wilberkihn053212',
 'kileygutmann007793',
 'clovistreutel166913',
 'lennysauer172606',
 'domenicaemmerich066334',
 'kristincole045552',
 'elenorrempel020666',
 'fideldach114224',
 'makenzieyundt001406',
 'lillyferry059245',
 'leraoreilly002404',
 'kianaschmitt004411',
 'patryan007546',
 'moshebernier100366',
 'rosaliatoy000359'

In [15]:
# Since we know that our partition key is /enrollmentYear, we know that any query that targets a single valid value for the 'enrollmentYear' property 
# will be a single partition query.

query = 'SELECT * FROM students s WHERE s.enrollmentYear = 2016'
query_results = query_documents(collection_link, query)
request_charge = client.last_response_headers['x-ms-request-charge']

print(f'Length of query results: {len(query_results)}')
print(f'RU: {request_charge}')


Length of query results: 2020
RU: 3.77


In [16]:
# If we want to execute a blanket query that will fan-out to all partitions, we simply can drop our WHERE clause that filters on a single valid
# value for our partition key path.
#
# Observe the Request Charge (in RU/s) for the executed query. You will notice that the charge is relatively greater for this query.

query = 'SELECT * FROM students s'
query_results = query_documents(collection_link, query)
request_charge = client.last_response_headers['x-ms-request-charge']

print(f'Length of query results: {len(query_results)}')
print(f'RU: {request_charge}')


Length of query results: 12500
RU: 3.57


In [17]:
# Observe the Request Charge (in RU/s) for the executed query. You will notice that the charge is greater than a single partition but far
# less than a fan-out across all partitions.

query = 'SELECT * FROM students s WHERE s.enrollmentYear IN (2015, 2016, 2017)'
query_results = query_documents(collection_link, query)
request_charge = client.last_response_headers['x-ms-request-charge']

print(f'Length of query results: {len(query_results)}')
print(f'RU: {request_charge}')


Length of query results: 6157
RU: 4.41


In [18]:
# To get the school-issued e-mail address, we will need to concatenate the 
# '@contoso.edu' string to the end of each alias. We can perform this action
# using the CONCAT built-in function.

query = 'SELECT CONCAT(s.studentAlias, "@contoso.edu") AS email FROM students s WHERE s.enrollmentYear = 2015'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2026
{'email': 'monserratlangworth070527@contoso.edu'}


In [19]:
# In most client-side applications, you likely would only need an array of 
# strings as opposed to an array of objects. We can use the VALUE keyword
# here to flatten our result set.

query = 'SELECT VALUE CONCAT(s.studentAlias, "@contoso.edu") FROM students s WHERE s.enrollmentYear = 2015'
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2026
'monserratlangworth070527@contoso.edu'


In [20]:
# In this query, we want to determine the current status of every
# student who enrolled in 2014. Our goal here is to eventually have a
# flattened, simple-to-understand view of every student and their current
# academic status.
# 
# You will quickly notice that the value representing the name of the
# student, using the CONCAT function, has a placeholder property name
# instead of a simple string.

query = '''
SELECT 
    CONCAT(s.firstName, " ", s.lastName), 
    s.academicStatus.warning, 
    s.academicStatus.suspension, 
    s.academicStatus.expulsion,
    s.enrollmentYear,
    s.projectedGraduationYear
FROM students s
WHERE s.enrollmentYear = 2014
'''
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2067
{'$1': 'Vincent White',
 'enrollmentYear': 2014,
 'expulsion': False,
 'projectedGraduationYear': 2018,
 'suspension': False,
 'warning': True}


In [21]:
# We will update our previous query by naming our property that uses
# a built-in function.

query = '''
SELECT 
    CONCAT(s.firstName, " ", s.lastName) AS name, 
    s.academicStatus.warning, 
    s.academicStatus.suspension, 
    s.academicStatus.expulsion,
    s.enrollmentYear,
    s.projectedGraduationYear
FROM students s
WHERE s.enrollmentYear = 2014
'''
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2067
{'enrollmentYear': 2014,
 'expulsion': False,
 'name': 'Vincent White',
 'projectedGraduationYear': 2018,
 'suspension': False,
 'warning': True}


In [22]:
# Another alternative way to specify the structure of our JSON document
# is to use the curly braces from JSON. At this point, we are defining
# the structure of the JSON result directly in our query.

query = '''
SELECT {
    "name": CONCAT(s.firstName, " ", s.lastName), 
    "isWarned": s.academicStatus.warning, 
    "isSuspended": s.academicStatus.suspension, 
    "isExpelled": s.academicStatus.expulsion,
    "enrollment": {
        "start": s.enrollmentYear,
        "end": s.projectedGraduationYear
    }
 } AS studentStatus
 FROM students s
 WHERE s.enrollmentYear = 2014
'''
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2067
{'studentStatus': {'enrollment': {'end': 2018, 'start': 2014},
                   'isExpelled': False,
                   'isSuspended': False,
                   'isWarned': True,
                   'name': 'Vincent White'}}


In [23]:
# If we want to “unwrap” our JSON data and flatten to a simple array of
# like-structured objects, we need to use the 'VALUE' keyword.

query = '''
SELECT VALUE {
    "name": CONCAT(s.firstName, " ", s.lastName), 
    "isWarned": s.academicStatus.warning, 
    "isSuspended": s.academicStatus.suspension, 
    "isExpelled": s.academicStatus.expulsion,
    "enrollment": {
        "start": s.enrollmentYear,
        "end": s.projectedGraduationYear
    }
 }
 FROM students s
 WHERE s.enrollmentYear = 2014
'''
query_results = query_documents(collection_link, query)

print(f'Length of query results: {len(query_results)}')
pprint(query_results[0])


Length of query results: 2067
{'enrollment': {'end': 2018, 'start': 2014},
 'isExpelled': False,
 'isSuspended': False,
 'isWarned': True,
 'name': 'Vincent White'}


In [24]:
# Notice that this query returns more than 3 items although the option 'maxItemCount' is specified to 3. 
# This is because the return value of 'CosmosClient.QueryItems' is converted to list in the definition
# of function 'query_documents'. The resulting list contains all items returned by the query

query = '''
SELECT VALUE {
    "name": CONCAT(s.firstName, " ", s.lastName), 
    "isWarned": s.academicStatus.warning, 
    "isSuspended": s.academicStatus.suspension, 
    "isExpelled": s.academicStatus.expulsion,
    "enrollment": {
        "start": s.enrollmentYear,
        "end": s.projectedGraduationYear
    }
 }
 FROM students s
 WHERE s.enrollmentYear = 2014
'''
query_options = {
    "enableCrossPartitionQuery": True,
    'maxItemCount': 3
}
query_iterable = query_documents(collection_link, query, query_options)

print(f'Length of query results: {len(query_iterable)}')
pprint(query_iterable)


Length of query results: 2067
[{'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Vincent White'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Dayana Greenfelder'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Camille Schulist'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Jessica Turcotte'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Stan Leannon'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Bertha Douglas'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': Fa

  'isWarned': False,
  'name': 'Mara Volkman'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Nettie Trantow'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Nathen Koepp'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Amaya Thompson'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Bailey Green'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Arnulfo Watsica'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Hallie Lockman'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'i

  'isSuspended': False,
  'isWarned': False,
  'name': 'Selina Jacobs'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Howard Considine'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Chance Crona'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Magdalen Roberts'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Bettye Hodkiewicz'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Akeem Paucek'},
 {'enrollment': {'end': 2023, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Kale Schamberger'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': Fa

  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Nestor Jacobson'},
 {'enrollment': {'end': 2023, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Earline Leffler'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Laura Rolfson'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Laverna Okuneva'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Mohammed Jast'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Scot Fadel'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Yvette Heathcote'},
 {'enrollment': {'end': 2024, 'start': 2014},
 

  'name': 'Renee Cummings'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Mckayla Kohler'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Carlie Bartell'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Ellsworth Robel'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Kenton Sawayn'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Peyton Herzog'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Kyra Wiegand'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'n

...

  'name': 'Rodolfo Leffler'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Elmira Veum'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Rylan Kuhn'},
 {'enrollment': {'end': 2023, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Nyasia Morar'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Santa Stiedemann'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Gianni Marquardt'},
 {'enrollment': {'end': 2023, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Trent Trantow'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'n

  'name': 'Golden Borer'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Itzel Hessel'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Leonie Ankunding'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Ansel Schultz'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Nigel Dicki'},
 {'enrollment': {'end': 2021, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Jarrell Becker'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Cassidy Franecki'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  '

  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Aisha Crooks'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Bradley Kuhlman'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Lafayette Runolfsdottir'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Myriam Kreiger'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Baylee Roberts'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Dangelo Franecki'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Rodrick Gorczany'},
 {'enrollment': {'end': 2022, 'sta

  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Eldridge Pouros'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Elmer Bogisich'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Easton Cassin'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Tanner Rolfson'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Aurore Koch'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Destiny Kuhlman'},
 {'enrollment': {'end': 2022, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Monte DuBuque'},
 {'enrollment': {'end': 2021, 'start': 2014},
  

  'isSuspended': False,
  'isWarned': False,
  'name': 'Freddy Kozey'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Stan Gleichner'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Rahsaan Parisian'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Olin Huel'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Sarina Heller'},
 {'enrollment': {'end': 2025, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Floy Flatley'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Daija Klein'},
 {'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspe

In [25]:
# To leverage the pagination capability from Azure Cosmos DB, the returned value from 'CosmosClient.QueryItems'
# should not be converted to 'list' type. The 'fetch_next_block' function is used to obtain the next page from
# the 'QueryIterable' object

query = '''
SELECT VALUE {
    "name": CONCAT(s.firstName, " ", s.lastName), 
    "isWarned": s.academicStatus.warning, 
    "isSuspended": s.academicStatus.suspension, 
    "isExpelled": s.academicStatus.expulsion,
    "enrollment": {
        "start": s.enrollmentYear,
        "end": s.projectedGraduationYear
    }
 }
 FROM students s
 WHERE s.enrollmentYear = 2014
'''
query_options = {
    "enableCrossPartitionQuery": True,
    'maxItemCount': 3
}
query_iterable = client.QueryItems(collection_link, query, query_options)

query_page = query_iterable.fetch_next_block()

print(f'Length of query results: {len(query_page)}')
pprint(query_page)

Length of query results: 3
[{'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Vincent White'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Dayana Greenfelder'},
 {'enrollment': {'end': 2024, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Camille Schulist'}]


In [26]:
# Calling 'fetch_next_block' again returns the next 3 items in the array.

query_page = query_iterable.fetch_next_block()

print(f'Length of query results: {len(query_page)}')
pprint(query_page)

Length of query results: 3
[{'enrollment': {'end': 2020, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Jessica Turcotte'},
 {'enrollment': {'end': 2019, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': False,
  'name': 'Stan Leannon'},
 {'enrollment': {'end': 2018, 'start': 2014},
  'isExpelled': False,
  'isSuspended': False,
  'isWarned': True,
  'name': 'Bertha Douglas'}]


In [27]:
def delete_database(id):
    print('Delete Database')

    try:
       database_link = 'dbs/' + id
       client.DeleteDatabase(database_link)

       print(f'Database with id "{id}" was deleted')

    except errors.HTTPFailure as e:
        if e.status_code == 404:
           print('A database with id "{id}" does not exist')
        else: 
            raise

In [28]:
# Clean up
delete_database(COSMOSDB_DB_ID)

Delete Database
Database with id "UniversityDatabase" was deleted
